# Skip-Gram

Import Functions provided on our Github

In [ ]:
run Word2Vec_Functions.ipynb

In [ ]:
import pickle

## Data

### Load the data

The data must be a codes sequence per patient: 

codes_par_patient = [[Code1, Code2, Code3], [Patient 2]]

In [ ]:
codes_par_patient = ...

### Generate a dictionnary of unique medical codes

Format {'code' : id}

In [ ]:
vocab = ...

## Gridsearch of hyperparamaters

Tested hyperparameters
- window_size: number of neighboors 
- embedding_size: dimension of the codes latent space
- batch_size: size of batchs
- num_negative_samples: number of randomly false neighboors

In [ ]:
window_size_list = [5, 10]
num_negative_samples_list = [5, 10]
batch_size_list = [250]
embedding_dim_list = [10, 20, 50, 100]

Training Parameters

In [ ]:
epochs = 40
lr = 1e-3

In [ ]:
df_hyperparameters = Gridsearch_Train_Word2Vec(codes_par_patient, vocab, epochs, lr, window_size_list, num_negative_samples_list, batch_size_list, embedding_dim_list)

Optimal hyperparameters

We keep the set of hyperparameters minimizing the validation loss

In [ ]:
print('Optimal window size :' + str(df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].window_size))
print('Optimal number of negative samples :' + str(df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].num_negative_samples))
print('Optimal dimension of the latent space :' + str(df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].embedding_dim))

## Training of the Codes Representations

#### Optimal Hyperparameters

In [ ]:
window_size = df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].window_size
num_negative_samples = df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].num_negative_samples
embedding_dim = df_hyperparameters.iloc[np.argmin(df_hyperparameters['Total_Loss'])].embedding_dim
batch_size = 250

#### Data Preparation

In [ ]:
word_pairs = Word2VecDataset(codes_par_patient, vocab, window_size, num_negative_samples=num_negative_samples)

#### Split of the dataset into a train and a test

In [ ]:
word_pairs_train, word_pairs_test= train_test_split(word_pairs, test_size=0.2, random_state=42)

#### Generate dataloaders

In [ ]:
dataloader = DataLoader(word_pairs_train, batch_size=250, shuffle=True)

#### Training Step

Paramètres

In [ ]:
nb_epoch = 200
learning_rate = 1e-5

In [ ]:
model = Train_Word2Vec(dataloader, vocab, embedding_dim, nb_epoch, learning_rate)

Saving the model

In [ ]:
torch.save(model.state_dict(), 'model.pth')

#### Evaluation

In [ ]:
dataloader_train, dataloader_test = DataLoader(word_pairs_train, batch_size=len(word_pairs_train), shuffle=True), DataLoader(word_pairs_test, batch_size=len(word_pairs_test), shuffle=True)                  

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for target, context, label in dataloader_train:
    target, context, label = target.to(device), context.to(device), label.to(device)
    output = model(target, context)
    label_tensor = label.clone().detach().view(label.size()[0], 1).to(dtype=torch.float)
    loss_train = nn.BCEWithLogitsLoss()(output, label_tensor)

In [ ]:
for target, context, label in dataloader_test:
    target, context, label = target.to(device), context.to(device), label.to(device)
    output = model(target, context)
    label_tensor = label.clone().detach().view(label.size()[0], 1).to(dtype=torch.float)
    loss_test = nn.BCEWithLogitsLoss()(output, label_tensor)

In [ ]:
print('Loss on train : ' + str(round(np.array(loss_train.cpu().detach()).item(),4)))
print('Loss on test : ' + str(round(np.array(loss_test.cpu().detach()).item(),4)))

## Get the Codes Representations

In [ ]:
code_representations = np.array(model.cpu().state_dict()['embeddings.weight'].detach().data)

Saving

In [ ]:
np.save('codes_representations', code_representations)

## Patient Representations

In [ ]:
patient_representation = []
for sentence in codes_par_patient :
    sentence_embeds = torch.zeros(embedding_dim)
    for code in sentence:
        word_embed = model.cpu().embeddings(torch.tensor(vocab[code]))
        sentence_embeds += word_embed
    patient_representation.append(sentence_embeds.detach().tolist())

Saving

In [ ]:
np.save('patients_embeddings', patient_representation)